In this notebook we will cover three main components that will help you write performant Julia code. We will have twenty mini case studies and each one will be tagged with one of these tags: 
- (1) Tools, 
- (2) Types, and 
- (3) Memory.

## 1️⃣ [Tools] Benchmarking
Throughout this notebook, we will benchmark code and compare timings between functions, and we will be using the [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) package.

Functionality worth noting:
- `@btime`
- `@benchmark`
- `BenchmarkTools.DEFAULT_PARAMETERS.samples`
- `BenchmarkTools.DEFAULT_PARAMETERS.seconds`

Let's try something out.

In [1]:
using BenchmarkTools
using LinearAlgebra
A = rand(1000,1000)
@btime norm(A)

  426.723 μs (1 allocation: 16 bytes)


577.1194584712035

In [2]:
function timing_from_a_function()
    B = rand(100,100)
    @btime norm(B)
end
timing_from_a_function()

UndefVarError: UndefVarError: B not defined

In [3]:
# notice which scope are you running the @btime macro from. 
# You will need to interpolate the variables you're passing to your functions.
function timing_from_a_function()
    B = rand(100,100)
    @btime norm($B)
end
timing_from_a_function()

  2.630 μs (0 allocations: 0 bytes)


57.56648914144387

In [4]:
A = rand(10_000,10_000)
b = @benchmark(norm($A))

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     66.541 ms (0.00% GC)
  median time:      68.270 ms (0.00% GC)
  mean time:        69.877 ms (0.00% GC)
  maximum time:     90.509 ms (0.00% GC)
  --------------
  samples:          72
  evals/sample:     1

In [5]:
b.times

72-element Array{Float64,1}:
 6.6540773e7
 6.6544951e7
 6.6802459e7
 6.6894026e7
 6.6932038e7
 6.6938535e7
 6.696567e7
 6.6976691e7
 6.7040825e7
 6.7060457e7
 6.7074917e7
 6.7095125e7
 6.7109001e7
 ⋮
 7.254496e7
 7.322845e7
 7.3403897e7
 7.3609716e7
 7.3779861e7
 7.4376496e7
 7.450855e7
 7.5501133e7
 7.6816566e7
 7.7050955e7
 8.5943142e7
 9.0509429e7

In [6]:
# you can force the number of samples
b = @benchmark(norm($A),samples=10)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     67.368 ms (0.00% GC)
  median time:      68.930 ms (0.00% GC)
  mean time:        70.622 ms (0.00% GC)
  maximum time:     79.535 ms (0.00% GC)
  --------------
  samples:          10
  evals/sample:     1

In [7]:
b.times

10-element Array{Float64,1}:
 6.7367765e7
 6.7665322e7
 6.8292852e7
 6.8684129e7
 6.8798195e7
 6.9060912e7
 7.0648926e7
 7.1236554e7
 7.4927986e7
 7.9534706e7

In [8]:
# you can also globally set the number of samples to run
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
b = @benchmark norm($A)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     66.323 ms (0.00% GC)
  median time:      67.675 ms (0.00% GC)
  mean time:        69.053 ms (0.00% GC)
  maximum time:     94.667 ms (0.00% GC)
  --------------
  samples:          73
  evals/sample:     1

In [9]:
b.times

73-element Array{Float64,1}:
 6.6323273e7
 6.6519851e7
 6.6694797e7
 6.6703923e7
 6.6877924e7
 6.6879407e7
 6.696749e7
 6.7020872e7
 6.70563e7
 6.706255e7
 6.7081524e7
 6.7096782e7
 6.7105498e7
 ⋮
 7.0193202e7
 7.0382941e7
 7.0410067e7
 7.1634035e7
 7.3559212e7
 7.432824e7
 7.446428e7
 7.4753271e7
 7.5695496e7
 8.2338899e7
 8.7254855e7
 9.4667437e7

Didn't we set the number of samples to 100?
Yes. But these will only run until a number of seconds is satisfied

In [11]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds

5.0

In [12]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 100

100

In [13]:
b = @benchmark norm($A)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     64.919 ms (0.00% GC)
  median time:      67.645 ms (0.00% GC)
  mean time:        68.107 ms (0.00% GC)
  maximum time:     75.331 ms (0.00% GC)
  --------------
  samples:          100
  evals/sample:     1

In [14]:
length(b.times)

100

Recap:
- BenchmarkTools is a great tool to do your benchmarking
- Don't forget to interpolate your variables when you are working with local scope
- You can control a number of things while benchmarking including the number of runs and the wait time until all runs finish. More here: https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md#benchmark-parameters

## 2️⃣ [Tools] Profiling
Profiling makes more sense when we run it from the terminal. We will demo the profiler there.

Here is the pipeline: 
- 1. Have a working solution of your work.
- 2. Start Julia with `--track-allocation=<setting>`.
- 3. Type `using Profile`.
- 4. Include the files needed to profile your code.
- 5. Run your code to make sure it's a running version and the results are as expected.
- 6. Profile.clear_malloc_data()
- 7. `@profile myfunc()`

Each line will have to its left the number of bytes allocated.

In [15]:
include("rosetta15_puzzle_game.jl")
puzzle15play()

This puzzle is not solvable.
+----+----+----+----+
| 8  | 10 | 1  | 3  |
| 14 | 7  | 5  | 12 |
| 9  | 11 | 15 | 4  |
| 6  | 13 | 2  |    |
+----+----+----+----+
Possible moves are: ["2", "4"], 0 to exit. Your move? =>  stdin> 0
Exiting the game...


## 3️⃣ [Types] Avoid abstract types
- Abstract type is something like Real
- Concrete type is something like Int64

First let's take a look at some of Julia's types

In [16]:
function _show_subtype_tree(mytype,printlevel)
    allsubtypes = subtypes(mytype)
    for cursubtype in allsubtypes
        print("\t"^printlevel)
        println("|___",cursubtype)
        printlevel += 1
        _show_subtype_tree(cursubtype,printlevel)
        printlevel -= 1
    end
end
function show_type_tree(T)
    println(T)
    _show_subtype_tree(T,0)
end
show_type_tree(Number)

Number
|___Complex
|___Real
	|___AbstractFloat
		|___BigFloat
		|___Float16
		|___Float32
		|___Float64
	|___AbstractIrrational
		|___Irrational
	|___Integer
		|___Bool
		|___Signed
			|___BigInt
			|___Int128
			|___Int16
			|___Int32
			|___Int64
			|___Int8
		|___Unsigned
			|___UInt128
			|___UInt16
			|___UInt32
			|___UInt64
			|___UInt8
	|___Rational


In [17]:
@show isconcretetype(Float64);
@show isconcretetype(Real);

isconcretetype(Float64) = true
isconcretetype(Real) = false


In [18]:
function record_games_won(ngames)
    games_won = []
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end
function record_games_won_v2(ngames)
    games_won = Int64[]
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end
record_games_won(2);
record_games_won_v2(2);

ntrials = 1000
@btime record_games_won(ntrials);
@btime record_games_won_v2(ntrials);

  23.329 μs (227 allocations: 11.73 KiB)
  19.015 μs (9 allocations: 8.33 KiB)


## 4️⃣ [Types] Avoid global scope
In Julia, you always want to try and put your code in functions (mainly when you care about performance). Global scope is anything in the REPL or here on a jupyter notebook. Local scope is something that is in a function. 

In [19]:
allgames = rand(ntrials)
function record_games_won_global()
    games_won = Int64[]
    for (curi,curgame) in enumerate(allgames)
        if curgame >= 0.5
            push!(games_won,curi)
        end
    end
    return games_won
end

function record_games_won_local(ntrials)
    allgames = rand(ntrials)
    games_won = Int64[]
    for (curi,curgame) in enumerate(allgames)
        if curgame >= 0.5
            push!(games_won,curi)
        end
    end
    return games_won
end

record_games_won_global();
record_games_won_local(ntrials);

@btime record_games_won_global();
@btime record_games_won_local(ntrials);
    

  159.376 μs (4499 allocations: 140.98 KiB)
  13.054 μs (10 allocations: 16.27 KiB)


If you **must** use global scope, define your variable with `const`. `const` here does not mean that the value you're declaring cannot change. It only means that the type of the value you are declaring cannot change. Let's look at this example:

In [20]:
const myglobalint = 1
myglobalint = 2
@show myglobalint

myglobalint = 2


2

In [21]:
myglobalint = 1.5

ErrorException: invalid redefinition of constant myglobalint

## 5️⃣ [Memory] Preallocate memory

In [22]:
function record_games_won_v2(ngames)
    games_won = Int64[]
    for i = 1:ngames
        r = rand()
        if r >= 0.5
            push!(games_won,i)
        end
    end
    return games_won
end

function record_games_won_preallocate(ntrials)
    allgames = rand(ntrials)
    games_won = Vector{Int64}(undef,ntrials)
    gi = 1
    for (curi,curgame) in enumerate(allgames)
        if curgame >= 0.5
            games_won[gi] = curi
            gi += 1
        end
    end
    return games_won[1:gi-1]
end


record_games_won_preallocate(ntrials);

@btime record_games_won_v2(ntrials);
@btime record_games_won_preallocate(ntrials);

  18.938 μs (9 allocations: 8.33 KiB)
  7.394 μs (3 allocations: 19.80 KiB)


## 6️⃣ [Memory] Use fused vector operations (broadcast)

In [23]:
function record_games_won_preallocate_fused(ntrials)
    allgames = rand(ntrials)
    games_won = findall(allgames.>=0.5)
    return games_won
end

record_games_won_preallocate_fused(ntrials);
@btime record_games_won_preallocate_fused(ntrials);

  3.314 μs (5 allocations: 16.32 KiB)


In [24]:
f(x) = 3x.^2 + 4x + 7x.^3;
x = rand(1000)
@btime f(x)
@btime f.(x)
;

  4.604 μs (6 allocations: 47.63 KiB)
  1.155 μs (4 allocations: 8.00 KiB)


## 7️⃣ [Memory] You don't need to "vectorize"

In [25]:
function find_hypotenuse_vectorized(b,h)
    return sqrt.(b.^2+h.^2)
end

function find_hypotenuse_forloop(b,h)
    accum_vec = similar(b)
    for i = 1:length(b)
        accum_vec[i] = sqrt(b[i]^2+h[i]^2)
    end
    return accum_vec
end

b = rand(ntrials)
h = rand(ntrials)
@btime find_hypotenuse_vectorized(b,h);
@btime find_hypotenuse_vectorized.(b,h);
@btime find_hypotenuse_forloop(b,h);

  4.573 μs (4 allocations: 31.75 KiB)
  1.683 μs (3 allocations: 7.98 KiB)
  2.296 μs (1 allocation: 7.94 KiB)


## 8️⃣ [Memory] Reuse memory

In [34]:
function find_sum_of_sqrt_vectors(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    for i = 1:nvectors
        # v = sqrt.(1:i)
        sumvector[i] = sum(sqrt.(1:i))
    end
    return sumvector
end

function find_sum_of_sqrt_vectors_reusemem(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    v = Vector{Float64}(undef,nvectors)
    for i = 1:nvectors
        v[1:i] .= sqrt.(1:i)
        sumvector[i] = sum(v)
        v .= 0
    end
    return sumvector
end

@btime find_sum_of_sqrt_vectors(ntrials);
@btime find_sum_of_sqrt_vectors_reusemem(ntrials);

  1.045 ms (1001 allocations: 3.97 MiB)
  866.446 μs (2 allocations: 15.88 KiB)


## 9️⃣ [Memory] Use `@view` when you don't need a copy of the data

In [35]:
using SparseArrays
using LinearAlgebra
A = sprand(500,500,0.1)
function set_sum(A,rowids,colids)
    s = sum(A[rowids,colids])
end
function set_sum_view(A,rowids,colids)
    s = sum(view(A,rowids,colids))
end

using Random
@btime set_sum(A,randperm(10), randperm(10))
@btime set_sum_view(A,randperm(10), randperm(10))

  3.177 μs (19 allocations: 5.83 KiB)
  2.179 μs (6 allocations: 448 bytes)


4.100851697845265

In [36]:
function find_sum_of_sqrt_vectors_copies(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    v = sqrt.(1:nvectors)
    for i = 1:nvectors
        sumvector[i] = sum(v[1:i])
    end
    return sumvector
end
@btime find_sum_of_sqrt_vectors_copies(ntrials);

  602.272 μs (1002 allocations: 3.98 MiB)


In [37]:
function find_sum_of_sqrt_vectors_views(nvectors)
    sumvector = Vector{Float64}(undef,nvectors)
    v = sqrt.(1:nvectors)
    for i = 1:nvectors
        sumvector[i] = sum(@view v[1:i])
    end
    return sumvector
end
@btime find_sum_of_sqrt_vectors_views(ntrials);

  69.300 μs (1002 allocations: 62.75 KiB)


## 1️⃣0️⃣ [Memory] Use inplace operations if you don't need to keep the original data.

In [41]:
function find_hypotenuse_vectorized(b,h)
    return sqrt.(b.^2+h.^2)
end

function find_hypotenuse_inplaceb!(b,h)
    for i = 1:length(b)
        b[i] = sqrt(b[i]^2+h[i]^2)
    end
end

b = rand(ntrials)
h = rand(ntrials)
@btime find_hypotenuse_vectorized(b,h);
@btime find_hypotenuse_vectorized.(b,h);
@btime find_hypotenuse_forloop(b,h);
@btime find_hypotenuse_inplaceb!(b,h);

  5.018 μs (4 allocations: 31.75 KiB)
  1.713 μs (3 allocations: 7.98 KiB)
  2.381 μs (1 allocation: 7.94 KiB)
  2.205 μs (0 allocations: 0 bytes)


## 1️⃣1️⃣ [Memory] Use iterators if you only want to access each element once

### What are iterators and why do we care about them?
- We create iterator objects when we don't want to store/create all the elements in an array at once.
- A quick example is a Fibonacci sequence: say you want to use the Fibonacci sequence numbers for a simple purpose but you don't necessarily care about storing all of them. You would want something like this:
```
fib_iterator = fib(n)
for i in fib_iterator
  #do something
end
```
- In the above iteration, you are not computing and storing all the fibonacci sequence numbers. Instead, we are just creating them on the fly
A lot of types in Julia are iteratable by default. A simple example is an array of numbers.
```
for i in rand(10)
  #do something
end
```
`rand(10)` returns a vector, and a vector is iteratable

In [42]:
struct fib_iterator
    n::Int
end

function Base.iterate(f::fib_iterator,state=(0,0,1))
    prev1,prev2,stepid = state
    # state the ending conditions first
    if stepid == 1
        return (1,(0,1,2))
    end
    if f.n < stepid
        return nothing
    end
    # else
    y = prev1+prev2
    stepid += 1
    return (y,(prev2,y,stepid))
end

function myfib(n)
    v = zeros(Int,n+1)
    v[1] = 1
    v[2] = 1
    for i = 3:n+1
        v[i] = v[i-1] + v[i-2]
    end
    return v
end

myfib (generic function with 1 method)

In [43]:
function test_iterator(n)
    f = fib_iterator(n)
    s = 0
    for i in f
        s += i
    end
end
function test_allocate(n)
    s = 0
    for i in myfib(n)
        s += i
    end
end
    
@btime test_iterator(10);
@btime test_allocate(10);

  9.692 ns (0 allocations: 0 bytes)
  67.756 ns (1 allocation: 176 bytes)


## 1️⃣2️⃣ [Memory] Access matrices by columns first

In [44]:
m = ntrials
n = 10000
A = rand(m,n)

function matrix_sum_rows(A)
    m,n = size(A)
    mysum = 0
    for i = 1:m # fix a row
        for j = 1:n # loop over cols
            mysum += A[i,j]
        end
    end
    return mysum
end

function matrix_sum_cols(A)
    m,n = size(A)
    mysum = 0
    for j = 1:n # fix a column
        for i = 1:m # loop over rows
            mysum += A[i,j]
        end
    end
    return mysum
end

function matrix_sum_index(A)
    m,n = size(A)
    mysum = 0
    for i = 1:m*n
        mysum += A[i]
    end
    return mysum
end
@btime matrix_sum_rows(A)
@btime matrix_sum_cols(A)
@btime matrix_sum_index(A);

  42.897 ms (1 allocation: 16 bytes)
  14.700 ms (1 allocation: 16 bytes)
  13.371 ms (1 allocation: 16 bytes)


## 1️⃣3️⃣ [Types] Write type stable code

In [45]:
function zero_or_val_stable(x)
    if x >= 0
        return x
    else
        return zero(typeof(x)) 
    end
end
@code_warntype zero_or_val_stable(0.2)

function zero_or_val(x)
    if x >= 0
        return x
    else
        return 0
    end
end
@code_warntype zero_or_val(0.2)

Variables
  #self#::Core.Compiler.Const(zero_or_val_stable, false)
  x::Float64

Body::Float64
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Main.typeof(x)::Core.Compiler.Const(Float64, false)
│   %5 = Main.zero(%4)::Core.Compiler.Const(0.0, false)
└──      return %5
Variables
  #self#::Core.Compiler.Const(zero_or_val, false)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


In [50]:
@code_warntype zero_or_val_stable(0.2)

Variables
  #self#::Core.Compiler.Const(zero_or_val_stable, false)
  x::Float64

Body::Float64
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Main.typeof(x)::Core.Compiler.Const(Float64, false)
│   %5 = Main.zero(%4)::Core.Compiler.Const(0.0, false)
└──      return %5


In [51]:
@code_warntype zero_or_val(0.2)

Variables
  #self#::Core.Compiler.Const(zero_or_val, false)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


## 1️⃣4️⃣ [Types] Type stability revisited

Let's take a look at the example below.

In [52]:
function square_plus_one(v::T) where T <:Number
    g = v*v
    return g+1
end

square_plus_one (generic function with 1 method)

In [53]:
v = rand()
typeof(v)

Float64

In [54]:
@code_warntype square_plus_one(v)

Variables
  #self#::Core.Compiler.Const(square_plus_one, false)
  v::Float64
  g::Float64

Body::Float64
1 ─      (g = v * v)
│   %2 = (g + 1)::Float64
└──      return %2


In [55]:
w = 5
typeof(w)

Int64

In [56]:
@code_warntype square_plus_one(w)

Variables
  #self#::Core.Compiler.Const(square_plus_one, false)
  v::Int64
  g::Int64

Body::Int64
1 ─      (g = v * v)
│   %2 = (g + 1)::Int64
└──      return %2


Great! In the above two examples, we were able to predict what the output will be. This is because:

```
function square_plus_one(v::T) where T <:Number
    g = v*v         # Type(T * T) ==> T
    return g+1      # Type(T + Int)) ==> "max" (T,Int)
end
```
Note that in both calls the return type was different, once Float64 and once Int64. But the function is still type stable.

In [58]:
function myfunction(i)
    if i%2==0
        i/2
    else
        3*i+1
    end
end
@code_warntype myfunction(2.2)

Variables
  #self#::Core.Compiler.Const(myfunction, false)
  i::Float64

Body::Float64
1 ─ %1 = (i % 2)::Float64
│   %2 = (%1 == 0)::Bool
└──      goto #3 if not %2
2 ─ %4 = (i / 2)::Float64
└──      return %4
3 ─ %6 = (3 * i)::Float64
│   %7 = (%6 + 1)::Float64
└──      return %7


Some handy tricks to help with type stability:

In [59]:
function zero_or_val_stable_promote(x)
    T = promote_type(typeof(x),Int)
    if x >= 0
        return T(x)
    else
        return T(0)
    end
end

function zero_or_val_stable_annotate(x)
    if x >= 0
        return x
    else
        return 0::typeof(x)
    end
end

function zero_or_val_stable_T(x::T) where T<:Real
    if x >= 0
        return x
    else
        return T(x)
    end
end

function zero_or_val_stable_zeroT(x)
    if x >= 0
        return x
    else
        return zero(typeof(x))
    end
end

function zero_or_val_stable_zerox(x)
    if x >= 0
        return x
    else
        return zero(x)
    end
end

function zero_or_val_stable_convert(x)
    if x >= 0
        return x
    else
        return convert(typeof(x),0)
    end
end

zero_or_val_stable_convert (generic function with 1 method)

In [60]:
@show w
@code_warntype zero_or_val_stable_convert(w)

w = 5
Variables
  #self#::Core.Compiler.Const(zero_or_val_stable_convert, false)
  x::Int64

Body::Int64
1 ─ %1 = (x >= 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Main.typeof(x)::Core.Compiler.Const(Int64, false)
│   %5 = Main.convert(%4, 0)::Core.Compiler.Const(0, false)
└──      return %5


Let's revisit the matrix sum code and write a type stable function.

In [61]:
function matrix_sum_index(A::Array{T,2}) where T
    m,n = size(A)
    mysum = 0
    for i = 1:m*n
        mysum += A[i]
    end
    return mysum
end

A = rand(10,10)
@code_warntype matrix_sum_index(A)

Variables
  #self#::Core.Compiler.Const(matrix_sum_index, false)
  A::Array{Float64,2}
  m::Int64
  @_4::Int64
  n::Int64
  mysum::Union{Float64, Int64}
  @_7::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Union{Float64, Int64}
1 ─ %1  = Main.size(A)::Tuple{Int64,Int64}
│   %2  = Base.indexed_iterate(%1, 1)::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(2, false)])
│         (m = Core.getfield(%2, 1))
│         (@_4 = Core.getfield(%2, 2))
│   %5  = Base.indexed_iterate(%1, 2, @_4::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(3, false)])
│         (n = Core.getfield(%5, 1))
│         (mysum = 0)
│   %8  = (m * n)::Int64
│   %9  = (1:%8)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_7 = Base.iterate(%9))
│   %11 = (@_7 === nothing)::Bool
│   %12 = Base.not_int(%11)::Bool
└──       goto #4 if not %12
2 ┄ %14 = @_7::Tuple{Int64,

In [67]:
function matrix_sum_index(A::Array{T,2}) where T
    m,n = size(A)
    mysum = T(0) # or eltype(A)
    for i in eachindex(A)
        mysum += A[i]
    end
    return mysum
end
A = rand(10,10)
@code_warntype matrix_sum_index(A)

Variables
  #self#::Core.Compiler.Const(matrix_sum_index, false)
  A::Array{Float64,2}
  m::Int64
  @_4::Int64
  n::Int64
  mysum::Float64
  @_7::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Float64
1 ─ %1  = Main.size(A)::Tuple{Int64,Int64}
│   %2  = Base.indexed_iterate(%1, 1)::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(2, false)])
│         (m = Core.getfield(%2, 1))
│         (@_4 = Core.getfield(%2, 2))
│   %5  = Base.indexed_iterate(%1, 2, @_4::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Int64,Int64}, Any[Int64, Core.Compiler.Const(3, false)])
│         (n = Core.getfield(%5, 1))
│         (mysum = ($(Expr(:static_parameter, 1)))(0))
│   %8  = Main.eachindex(A)::Base.OneTo{Int64}
│         (@_7 = Base.iterate(%8))
│   %10 = (@_7 === nothing)::Bool
│   %11 = Base.not_int(%10)::Bool
└──       goto #4 if not %11
2 ┄ %13 = @_7::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%13, 1))
│   %15 = Core.

## 1️⃣5️⃣ [Types] Avoid fields with abstract types

In [68]:
mutable struct oneCube
    length
    width
    height
end
volume(c::oneCube) = c.length*c.width*c.height

mutable struct oneCube_Real
    length::Real
    width::Real
    height::Real
end
volume(c::oneCube_Real) = c.length*c.width*c.height

mutable struct oneCube_parametric_typed{T <: Real}
    length::T
    width::T
    height::T
end
volume(c::oneCube_parametric_typed) = c.length*c.width*c.height

c1 = oneCube(1.1,1.2,1.3)
c2 = oneCube_Real(1.1,1.2,1.3)
c3 = oneCube_parametric_typed(1.1,1.2,1.3)
@show volume(c1) == volume(c2) == volume(c3)


@btime volume(c1) # not typed
@btime volume(c2) # typed float
@btime volume(c3) # typed parametric



volume(c1) == volume(c2) == volume(c3) = true
  37.191 ns (1 allocation: 16 bytes)
  39.945 ns (1 allocation: 16 bytes)
  29.547 ns (1 allocation: 16 bytes)


1.7160000000000002

In [69]:
@show typeof(c1)
@show typeof(c2)
@show typeof(c3)

typeof(c1) = oneCube
typeof(c2) = oneCube_Real
typeof(c3) = oneCube_parametric_typed{Float64}


oneCube_parametric_typed{Float64}

In [70]:
c2.height = 1
typeof(c2.height)

Int64

In [71]:
c3.height = 1
typeof(c3.height)

Float64

In [72]:
@code_warntype volume(c1)
@code_warntype volume(c2)
@code_warntype volume(c3)

Variables
  #self#::Core.Compiler.Const(volume, false)
  c::oneCube

Body::Any
1 ─ %1 = Base.getproperty(c, :length)::Any
│   %2 = Base.getproperty(c, :width)::Any
│   %3 = Base.getproperty(c, :height)::Any
│   %4 = (%1 * %2 * %3)::Any
└──      return %4
Variables
  #self#::Core.Compiler.Const(volume, false)
  c::oneCube_Real

Body::Any
1 ─ %1 = Base.getproperty(c, :length)::Real
│   %2 = Base.getproperty(c, :width)::Real
│   %3 = Base.getproperty(c, :height)::Real
│   %4 = (%1 * %2 * %3)::Any
└──      return %4
Variables
  #self#::Core.Compiler.Const(volume, false)
  c::oneCube_parametric_typed{Float64}

Body::Float64
1 ─ %1 = Base.getproperty(c, :length)::Float64
│   %2 = Base.getproperty(c, :width)::Float64
│   %3 = Base.getproperty(c, :height)::Float64
│   %4 = (%1 * %2 * %3)::Float64
└──      return %4


In [124]:
mutable struct mynewReal
    val::Real
end

In [129]:
# follow up:
function mysumc(v::Vector{mynewReal})
    s = zero(eltype(v[1].val))
    for i = 1:length(v)
        s += v[i].val
    end
    return s
end

function mysumT(v::Vector{T}) where T <:Real
    s = zero(eltype(v))
    for i = 1:length(v)
        s += v[i]
    end
    return s
end

v = rand(10);


In [130]:
vNewReal = mynewReal.(v)
mysumc(vNewReal)

4.171539403926762

In [131]:
mysumT(v)

4.171539403926762

In [132]:
@code_warntype mysumc(vNewReal)

Variables
  #self#::Core.Compiler.Const(mysumc, false)
  v::Array{mynewReal,1}
  s::Any
  @_4::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64

Body::Any
1 ─ %1  = Base.getindex(v, 1)::mynewReal
│   %2  = Base.getproperty(%1, :val)::Real
│   %3  = Main.eltype(%2)::Type{#s69} where #s69<:Real
│         (s = Main.zero(%3))
│   %5  = Main.length(v)::Int64
│   %6  = (1:%5)::Core.Compiler.PartialStruct(UnitRange{Int64}, Any[Core.Compiler.Const(1, false), Int64])
│         (@_4 = Base.iterate(%6))
│   %8  = (@_4 === nothing)::Bool
│   %9  = Base.not_int(%8)::Bool
└──       goto #4 if not %9
2 ┄ %11 = @_4::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%11, 1))
│   %13 = Core.getfield(%11, 2)::Int64
│   %14 = s::Any
│   %15 = Base.getindex(v, i)::mynewReal
│   %16 = Base.getproperty(%15, :val)::Real
│         (s = %14 + %16)
│         (@_4 = Base.iterate(%6, %13))
│   %19 = (@_4 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #4 if not %20
3 ─       goto

In [96]:
@code_warntype mysumT(v)

Variables
  #self#::Core.Compiler.Const(mysumT, false)
  v::Array{Float64,1}
  s::Type{Float64}
  @_4::Union{Nothing, Tuple{Float64,Int64}}
  vi::Float64

Body::Type{Float64}
1 ─      (s = Main.eltype(v))
│   %2 = v::Array{Float64,1}
│        (@_4 = Base.iterate(%2))
│   %4 = (@_4 === nothing)::Bool
│   %5 = Base.not_int(%4)::Bool
└──      goto #3 if not %5
2 ─ %7 = @_4::Tuple{Float64,Int64}::Tuple{Float64,Int64}
│        (vi = Core.getfield(%7, 1))
│        Core.getfield(%7, 2)
│        (s = s::Core.Compiler.Const(Float64, false) + vi)
│        Core.Compiler.Const(:(@_4 = Base.iterate(%2, %9)), false)
│        Core.Compiler.Const(:(@_4 === nothing), false)
│        Core.Compiler.Const(:(Base.not_int(%12)), false)
│        Core.Compiler.Const(:(unless %13 goto %16), false)
└──      Core.Compiler.Const(:(goto %7), false)
3 ┄      return s::Core.Compiler.Const(Float64, false)


In [135]:
# one more follow up: Avoid containers with abstract type parameters
h = Real[]
v = Float64[]

0-element Array{Float64,1}

## 1️⃣6️⃣ [Memory] Avoid allocating memory multiple times

In [136]:
function fmedian(n)
    v = zeros(Int64,n)
    for i = 1:n
        rand() >= 0.5 ? v[i] = 1 : nothing
    end
    return v
end

function fquarter(n)
    v = zeros(Int64,n)
    for i = 1:n
        rand() >= 0.25 ? v[i] = 1 : nothing
    end
    return v
end

function f3quarters(n)
    v = zeros(Int64,n)
    for i = 1:n
        rand() >= 0.75 ? v[i] = 1 : nothing
    end
    return v
end

function statistical_distribution(n)
    f1 = mean(fmedian(n))
    f2 = mean(fquarter(n))
    f3 = mean(f3quarters(n))
    return f1,f2,f3
end

statistical_distribution(1000)

(0.504, 0.768, 0.243)

In [137]:
function fmedian!(v,n)
    for i = 1:n
        rand() >= 0.5 ? v[i] = 1 : nothing
    end
    return v
end

function fquarter!(v,n)
    for i = 1:n
        rand() >= 0.25 ? v[i] = 1 : nothing
    end
    return v
end

function f3quarters!(v,n)
    for i = 1:n
        rand() >= 0.75 ? v[i] = 1 : nothing
    end
    return v
end

function statistical_distribution_predefinev(n)
    v = zeros(Int64,n)
    f1 = mean(fmedian!(v,n))
    v .= 0
    f2 = mean(fquarter!(v,n))
    v .= 0
    f3 = mean(f3quarters!(v,n))
    return f1,f2,f3
end

statistical_distribution_predefinev(1000)

(0.506, 0.743, 0.244)

In [138]:
@btime statistical_distribution(ntrials)
@btime statistical_distribution_predefinev(ntrials)

  37.297 μs (4 allocations: 23.84 KiB)
  40.902 μs (2 allocations: 7.97 KiB)


(0.486, 0.756, 0.249)

## 1️⃣7️⃣ [Memory] Use multiple dispatch instead of multiple conditionals

In [139]:
mutable struct square{T <: Real}
    length::T
    width::T
end

mutable struct circle{T <: Real}
    radius::T
end

function area_no_dispatch(geometric_shape)
    if isa(geometric_shape,square)
        return geometric_shape.length*geometric_shape.width
    elseif isa(geometric_shape,circle)
        return pi*geometric_shape.radius^2
    else
        error("invalid argument")
    end
end

area(c::circle) = pi*c.radius^2
area(s::square) = s.length*s.width

@btime area_no_dispatch(circle(2))
@btime area(circle(2))

  1.827 ns (0 allocations: 0 bytes)
  1.521 ns (0 allocations: 0 bytes)


12.566370614359172

## 1️⃣8️⃣ [Memory] StaticVectors can be useful when you know the length of the vector

In [140]:
# For each car, we'd like to store: make, miles, and number of previous owners. 
# We'll see what happens when we store them in a vector vs a static vector

mutable struct mutableCar
    description::Vector{Int}
    name::String
end

struct immutableCar
    description::Vector{Int}
    name::String
end

using StaticArrays: SVector

struct svectorCar
    x::SVector{3, Int}
    name::String
end

┌ Info: Precompiling StaticArrays [90137ffa-7385-5640-81b9-e52037218182]
└ @ Base loading.jl:1260


In [142]:
function test_mutable()
    for i in 1:1000
       x = mutableCar([rand(Int), rand(Int),rand(Int)],"Honda")
    end
end

function test_immutable()
    for i in 1:1000
       x = immutableCar([rand(Int), rand(Int),rand(Int)],"Honda")
    end
end

function test_SVector()
    for i in 1:1000
       x = svectorCar(SVector(rand(Int), rand(Int),rand(Int)),"Honda")
    end
end


@btime test_mutable()
@btime test_immutable()
@btime test_SVector()

  70.267 μs (1000 allocations: 109.38 KiB)
  69.363 μs (1000 allocations: 109.38 KiB)
  31.453 μs (0 allocations: 0 bytes)


## 1️⃣9️⃣ [Tools] Some macros that be handy - handle with caution!
Some macros that can come handy: `@inbounds`, `@fastmath`, `@inline`, `@simd`

In [143]:
?@inbounds

```
@inbounds(blk)
```

Eliminates array bounds checking within expressions.

In the example below the in-range check for referencing element `i` of array `A` is skipped to improve performance.

```julia
function sum(A::AbstractArray)
    r = zero(eltype(A))
    for i = 1:length(A)
        @inbounds r += A[i]
    end
    return r
end
```

!!! warning
    Using `@inbounds` may return incorrect results/crashes/corruption for out-of-bounds indices. The user is responsible for checking it manually. Only use `@inbounds` when it is certain from the information locally available that all accesses are in bounds.



In [144]:
?@fastmath

```
@fastmath expr
```

Execute a transformed version of the expression, which calls functions that may violate strict IEEE semantics. This allows the fastest possible operation, but results are undefined – be careful when doing this, as it may change numerical results.

This sets the [LLVM Fast-Math flags](http://llvm.org/docs/LangRef.html#fast-math-flags), and corresponds to the `-ffast-math` option in clang. See [the notes on performance annotations](@ref man-performance-annotations) for more details.

# Examples

```jldoctest
julia> @fastmath 1+2
3

julia> @fastmath(sin(3))
0.1411200080598672
```


In [145]:
?@inline

```
@inline
```

Give a hint to the compiler that this function is worth inlining.

Small functions typically do not need the `@inline` annotation, as the compiler does it automatically. By using `@inline` on bigger functions, an extra nudge can be given to the compiler to inline it. This is shown in the following example:

```julia
@inline function bigfunction(x)
    #=
        Function Definition
    =#
end
```


In [146]:
?@simd

```
@simd
```

Annotate a `for` loop to allow the compiler to take extra liberties to allow loop re-ordering

!!! warning
    This feature is experimental and could change or disappear in future versions of Julia. Incorrect use of the `@simd` macro may cause unexpected results.


The object iterated over in a `@simd for` loop should be a one-dimensional range. By using `@simd`, you are asserting several properties of the loop:

  * It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
  * Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In many cases, Julia is able to automatically vectorize inner for loops without the use of `@simd`. Using `@simd` gives the compiler a little extra leeway to make it possible in more situations. In either case, your inner loop should have the following properties to allow vectorization:

  * The loop must be an innermost loop
  * The loop body must be straight-line code. Therefore, [`@inbounds`](@ref) is   currently needed for all array accesses. The compiler can sometimes turn   short `&&`, `||`, and `?:` expressions into straight-line code if it is safe   to evaluate all operands unconditionally. Consider using the [`ifelse`](@ref)   function instead of `?:` in the loop if it is safe to do so.
  * Accesses must have a stride pattern and cannot be "gathers" (random-index   reads) or "scatters" (random-index writes).
  * The stride should be unit stride.

!!! note
    The `@simd` does not assert by default that the loop is completely free of loop-carried memory dependencies, which is an assumption that can easily be violated in generic code. If you are writing non-generic code, you can use `@simd ivdep for ... end` to also assert that:


  * There exists no loop-carried memory dependencies
  * No iteration ever waits on a previous iteration to make forward progress.


In [150]:
function new_sum(myvec)
    s = zero(eltype(myvec))
    for i = 1:length(myvec)
        s += myvec[i]
    end
    return s
end

function new_sum_inbounds(myvec)
    s = zero(eltype(myvec))
    for i = 1:length(myvec)
        @inbounds s += myvec[i]
    end
    return s
end

myvec = rand(Int,1000)
@btime new_sum_inbounds(myvec)
@btime new_sum(myvec)

  94.441 ns (1 allocation: 16 bytes)
  754.344 ns (1 allocation: 16 bytes)


7076626732405629251

In [151]:
function axpy!(a,x,y)
    @inbounds for i=1:length(x)
        y[i] = y[i]+a*x[i] 
    end
end

function axpyNoInbounds!(a,x,y)
    for i=1:length(x)
        y[i] = y[i]+a*x[i] 
    end
end

x = rand(ntrials)
y = rand(ntrials)
@btime axpy!(1,x,y)
@btime axpyNoInbounds!(1,x,y)

  162.238 ns (0 allocations: 0 bytes)
  1.109 μs (0 allocations: 0 bytes)


In [152]:
function new_sum_inbounds_WRONG(myvec::Vector{T}) where T<:Real
    s = 0
    @inbounds for i = 1:length(myvec)+1
        s += myvec[i]
    end
    return s
end
@btime new_sum_inbounds_WRONG(myvec)

  90.207 ns (1 allocation: 16 bytes)


7076626732405629243

Careful ordering of floating-point operations can be helpful.

In [156]:
const xx = 1
function testfastmath(i)
    y = i+xx+4
end
@code_llvm testfastmath(1.2)


;  @ In[156]:3 within `testfastmath'
define double @julia_testfastmath_20031(double) {
top:
; ┌ @ operators.jl:529 within `+' @ promotion.jl:311 @ float.jl:401
   %1 = fadd double %0, 1.000000e+00
   %2 = fadd double %1, 4.000000e+00
; └
  ret double %2
}


In [157]:
function testfastmath_applied(i)
    @fastmath y = i+xx+4
end
@code_llvm testfastmath_applied(1.2)


;  @ In[157]:2 within `testfastmath_applied'
define double @julia_testfastmath_applied_20035(double) {
top:
; ┌ @ fastmath.jl:263 within `add_fast' @ fastmath.jl:167 @ fastmath.jl:161
   %1 = fadd fast double %0, 5.000000e+00
; └
  ret double %1
}


`@inline` replaces a called function with its body.

In [158]:
@inline f(x,y) = div(x,y)*y 
g(x) = f(x,2)

g (generic function with 1 method)

In [159]:
@code_typed g(1)

CodeInfo(
1 ─ %1 = Base.checked_sdiv_int(x, 2)::Int64
│   %2 = Base.mul_int(%1, 2)::Int64
└──      return %2
) => Int64

In [160]:
@noinline f2(x,y) = div(x,y)*y 
g2(x) = f2(x,2)
@code_typed g2(1)

CodeInfo(
1 ─ %1 = invoke Main.f2(_2::Int64, 2::Int64)::Int64
└──      return %1
) => Int64

In [161]:
@code_typed f2(1,2)

CodeInfo(
1 ─ %1 = Base.checked_sdiv_int(x, y)::Int64
│   %2 = Base.mul_int(%1, y)::Int64
└──      return %2
2 ─      $(Expr(:meta, :noinline))
) => Int64

Inlining can be a slight win here. But usually the compiler is smart with `@inline` -- you may only need to use it with big functions.

## 2️⃣0️⃣ [Tools] If you want to see low level versions of your code.

In [162]:
myadd1(x) = x+1

myadd1 (generic function with 1 method)

In [163]:
@code_lowered myadd1(1)

CodeInfo(
1 ─ %1 = x + 1
└──      return %1
)

In [164]:
@code_warntype myadd1(1)

Variables
  #self#::Core.Compiler.Const(myadd1, false)
  x::Int64

Body::Int64
1 ─ %1 = (x + 1)::Int64
└──      return %1


In [165]:
@code_typed myadd1(1)

CodeInfo(
1 ─ %1 = Base.add_int(x, 1)::Int64
└──      return %1
) => Int64

In [166]:
@code_llvm myadd1(1)


;  @ In[162]:1 within `myadd1'
define i64 @julia_myadd1_20196(i64) {
top:
; ┌ @ int.jl:53 within `+'
   %1 = add i64 %0, 1
; └
  ret i64 %1
}


In [167]:
@code_native myadd1(1)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[162]:1 within `myadd1'
; │┌ @ In[162]:1 within `+'
	leaq	1(%rdi), %rax
; │└
	retq
	nopw	%cs:(%rax,%rax)
	nop
; └
